In [3]:
#Imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from io import StringIO
from typing import List, IO

In [4]:
# Set up Selenium
service: Service = Service()
options: webdriver.ChromeOptions = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no GUI)
options.add_argument('--no-sandbox')  # Bypass OS security model, needed for running in Docker
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
driver: webdriver.Chrome = webdriver.Chrome(service=service, options=options)

In [5]:
# URL to scrape
url: str = "https://www.fangraphs.com/standings/playoff-odds"

In [6]:
# Navigate to the page
driver.get(url)

In [7]:
# Wait for the tables to load
wait: WebDriverWait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'playoff-odds-table')))

[<selenium.webdriver.remote.webelement.WebElement (session="268f11edfb0d73151af23280d7a85a57", element="f.313291DEA46B2472D8FC79F930AFF7BD.d.28AD79CDA53D6BC2B566C1AB6A809FE2.e.85")>,
 <selenium.webdriver.remote.webelement.WebElement (session="268f11edfb0d73151af23280d7a85a57", element="f.313291DEA46B2472D8FC79F930AFF7BD.d.28AD79CDA53D6BC2B566C1AB6A809FE2.e.86")>,
 <selenium.webdriver.remote.webelement.WebElement (session="268f11edfb0d73151af23280d7a85a57", element="f.313291DEA46B2472D8FC79F930AFF7BD.d.28AD79CDA53D6BC2B566C1AB6A809FE2.e.87")>,
 <selenium.webdriver.remote.webelement.WebElement (session="268f11edfb0d73151af23280d7a85a57", element="f.313291DEA46B2472D8FC79F930AFF7BD.d.28AD79CDA53D6BC2B566C1AB6A809FE2.e.88")>,
 <selenium.webdriver.remote.webelement.WebElement (session="268f11edfb0d73151af23280d7a85a57", element="f.313291DEA46B2472D8FC79F930AFF7BD.d.28AD79CDA53D6BC2B566C1AB6A809FE2.e.89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="268f11edfb0d73151af23280d

In [8]:
# Get the page source and parse with BeautifulSoup
soup: BeautifulSoup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
# Find all tables with class "playoff-odds-table"
tables: List[BeautifulSoup] = soup.find_all('table', class_='playoff-odds-table')

In [10]:
# Close the Selenium driver
driver.quit()

In [21]:
unique_tables = set(tables)

In [22]:
df_list: List[pd.DataFrame]
for index, table in enumerate(unique_tables):
    html_str: str = str(table)
    html_io: IO[str] = StringIO(html_str)
    df_list.append(pd.read_html(html_io))

In [26]:
print(df_list[3])

[        Unnamed: 0_level_0 2024 Results                   2024 Projections  \
        Unnamed: 0_level_1 2024 Results                  [+]Regular Season   
               TeamNL East            W   L     W%    GB            Proj W   
0    PhilliesPhillies38-18           38  18  0.679   0.0              94.3   
1        BravesBraves31-21           31  21  0.596   5.0              92.8   
2            MetsMets22-32           22  32  0.407  15.0              77.0   
3  NationalsNationals24-29           24  29  0.453  12.5              70.5   
4      MarlinsMarlins19-37           19  37  0.339  19.0              68.3   

                                           ...                       \
                                           ...        [+]Postseason   
  Proj L ROS W% Strength of Sched Win Div  ... Unnamed: 103_level_2   
0   67.7  0.531             0.506   56.3%  ...                  NaN   
1   69.2  0.562             0.492   43.5%  ...                  NaN   
2   85.0  0.509    